# [Memory](https://langchain-ai.github.io/langgraph/concepts/memory/)
- LangGraph의 Memory는 AI가 대화 내용을 기억할 수 있게 해주는 기능입니다. 
- 사람이 대화할 때 이전에 말한 내용을 기억하는 것처럼, AI도 과거 대화를 기억해서 더 자연스러운 대화를 할 수 있게 해줍니다.


## 왜 Memory가 필요한가요?
> Memory가 없다면:
```text
    사용자: "내 이름은 김철수야"
    AI: "안녕하세요!"

    사용자: "내 이름이 뭐였지?"
    AI: "죄송해요, 모르겠습니다"
```
> Memory가 있다면:
```text
    사용자: "내 이름은 김철수야"
    AI: "안녕하세요 김철수님!"

    사용자: "내 이름이 뭐였지?"
    AI: "김철수님이라고 하셨어요!"
```


# LangGraph Memory 예시들

처음 배우는 학생들을 위한 단계별 예시를 준비했습니다!


In [1]:
# 필요한 라이브러리 import
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

True

## 1. Memory가 적용되지 않은 LangGraph 예시
**이름을 기억하지 못하는 챗봇**

### 1. AI 모델 설정

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 2. 그래프 노드 정의 (Memory 없음)

In [3]:
from langgraph.graph import MessagesState

def chat_node(state: MessagesState):
    # 현재 메시지만 처리 (이전 대화 기억 안함)
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

### 3. 그래프 생성

In [4]:
from langgraph.graph import StateGraph, MessagesState, START, END

graph = StateGraph(MessagesState)
graph.add_node("chat", chat_node)
graph.add_edge(START, "chat")
graph.add_edge("chat", END)

### 4. 그래프 컴파일 (Memory 없음)

In [5]:
app_without_memory = graph.compile()

print("Memory 없는 챗봇이 준비되었습니다!")

Memory 없는 챗봇이 준비되었습니다!


### 5. 테스트 

In [6]:
from langchain_core.messages import HumanMessage

# 첫 번째 대화
print("사용자: '내 이름은 김철수야'")
result1 = app_without_memory.invoke({"messages": [HumanMessage(content="내 이름은 김철수야")]})
print(f"AI: {result1['messages'][-1].content}")


사용자: '내 이름은 김철수야'
AI: 안녕하세요, 김철수님! 어떻게 도와드릴까요?


In [7]:
# 두 번째 대화 (새로운 대화)
print("사용자: '내 이름이 뭐였지?'")
result2 = app_without_memory.invoke({"messages": [HumanMessage(content="내 이름이 뭐였지?")]})
print(f"AI: {result2['messages'][-1].content}")

print("\n결과: AI가 이전 대화를 기억하지 못해요!")

사용자: '내 이름이 뭐였지?'
AI: 죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 당신의 이름을 알려주시면 그에 맞춰 대화할 수 있습니다!

결과: AI가 이전 대화를 기억하지 못해요!


## 2. Memory가 적용된 LangGraph 예시 (MemorySaver)
**이름을 기억하는 똑똑한 챗봇**


### 1. AI 모델 설정

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 2. 그래프 노드 정의 (Memory 적용)

In [9]:
from langgraph.graph import MessagesState

def chat_node_with_memory(state: MessagesState):
    # 모든 이전 메시지들과 함께 처리 (Memory가 자동으로 관리)
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

### 3. 그래프 생성

In [10]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver

graph_with_memory = StateGraph(MessagesState)
graph_with_memory.add_node("chat", chat_node_with_memory)
graph_with_memory.add_edge(START, "chat")
graph_with_memory.add_edge("chat", END)

### 4. Memory 저장소 설정 (여기가 핵심!)

In [11]:
memory = MemorySaver()

app_with_memory = graph_with_memory.compile(checkpointer=memory)

print("Memory가 적용된 챗봇이 준비되었습니다!")

Memory가 적용된 챗봇이 준비되었습니다!


### 5. 테스트 

In [12]:
from langchain_core.messages import HumanMessage
import uuid

# 중요: 같은 thread_id를 사용해야 대화가 연결됩니다!
memory_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": memory_id}}

print("=== Memory가 적용된 챗봇 테스트 ===")
print(f"config: {config}")

=== Memory가 적용된 챗봇 테스트 ===
config: {'configurable': {'thread_id': 'dfb15440-6dc3-48f5-9f93-5bd2b492205e'}}


In [13]:
# 첫 번째 대화
print("사용자: '내 이름은 김철수야'")
result1 = app_with_memory.invoke(
    {"messages": [HumanMessage(content="내 이름은 김철수야")]}, 
    config=config  # 여기가 중요!
)
print(f"AI: {result1['messages'][-1].content}")

사용자: '내 이름은 김철수야'
AI: 안녕하세요, 김철수님! 어떻게 도와드릴까요?


In [14]:
# 두 번째 대화 (같은 thread_id 사용)
print("사용자: '내 이름이 뭐였지?'")
result2 = app_with_memory.invoke(
    {"messages": [HumanMessage(content="내 이름이 뭐였지?")]}, 
    config=config  # 같은 config 사용!
)
print(f"AI: {result2['messages'][-1].content}")

print("\n결과: AI가 이전 대화를 기억합니다!")

사용자: '내 이름이 뭐였지?'
AI: 당신의 이름은 김철수입니다. 다른 질문이 있으신가요?

결과: AI가 이전 대화를 기억합니다!


## 3. ConversationBufferWindowMemory 예시
**최근 N개 메시지만 기억하는 효율적인 챗봇**

긴 대화에서는 모든 내용을 기억하면 메모리가 부족할 수 있어요. 그래서 최근 몇 개의 메시지만 기억하는 방법을 사용합니다!


### 1. AI 모델 설정

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 2. 윈도우 메모리 노드 정의

In [16]:
from langgraph.graph import MessagesState

def chat_node_with_window(state: MessagesState, window_size=4):
    """최근 window_size 개의 메시지만 유지하는 함수"""
    messages = state["messages"]
    
    # 메시지가 너무 많으면 최근 것만 선택
    if len(messages) > window_size:
        # 시스템 메시지는 항상 유지하고, 나머지는 최근 것만
        system_messages = [msg for msg in messages if msg.type == "system"]
        other_messages = [msg for msg in messages if msg.type != "system"]
        
        # 최근 메시지만 선택
        recent_messages = other_messages[-window_size:]
        windowed_messages = system_messages + recent_messages
    else:
        windowed_messages = messages
    
    response = llm.invoke(windowed_messages)
    return {"messages": [response]}

### 3. 그래프 생성

In [17]:
from langgraph.graph import StateGraph, MessagesState, START, END

graph_window = StateGraph(MessagesState)
graph_window.add_node("chat", chat_node_with_window)
graph_window.add_edge(START, "chat")
graph_window.add_edge("chat", END)

### 4. 컴파일

In [18]:
from langgraph.checkpoint.memory import MemorySaver 

memory_window = MemorySaver()
app_with_window = graph_window.compile(checkpointer=memory_window)

print("Window Memory 챗봇이 준비되었습니다! (최근 4개 메시지만 기억)")

Window Memory 챗봇이 준비되었습니다! (최근 4개 메시지만 기억)


### 5. 테스트 

In [19]:
from langchain_core.messages import HumanMessage
import uuid

# 중요: 같은 thread_id를 사용해야 대화가 연결됩니다!
memory_id = str(uuid.uuid4())
# Window Memory 테스트: 오래된 정보를 기억할까요?
config_window = {"configurable": {"thread_id": memory_id}}

print("=== Window Memory 챗봇 테스트 ===")
print("많은 정보를 말해보고, 나중에 오래된 정보를 기억하는지 확인해보겠습니다!\n")

=== Window Memory 챗봇 테스트 ===
많은 정보를 말해보고, 나중에 오래된 정보를 기억하는지 확인해보겠습니다!



In [20]:
# 여러 정보 입력
messages_to_test = [
    "내 이름은 김철수야",
    "나는 30살이야", 
    "내 이름이 뭐였지?", # 기억함 
    "서울에 살고 있어",
    "취미는 축구야",
    "좋아하는 색깔은 파란색이야",
    "어제 영화를 봤어",
    "오늘은 운동을 했어",
    "내 이름이 뭐였지?",  # 오래전 정보 (기억 못할 수도 있음)
    "어제 뭐했다고 했지?"  # 최근 정보 (기억해야 함)
]

for i, msg in enumerate(messages_to_test, 1):
    print(f"{i}번째 대화: '{msg}'")
    result = app_with_window.invoke(
        {"messages": [HumanMessage(content=msg)]}, 
        config=config_window
    )
    print(f"AI: {result['messages'][-1].content}")
    print("-" * 40)

print("\n결과: 최근 4개 메시지만 기억해서 메모리가 효율적!")
print("오래된 정보(이름)는 잊어버릴 수 있지만, 최근 정보(어제 한 일)는 기억해요!")

1번째 대화: '내 이름은 김철수야'
AI: 안녕하세요, 김철수님! 어떻게 도와드릴까요?
----------------------------------------
2번째 대화: '나는 30살이야'
AI: 30살이시군요! 이 나이에 특별히 하고 싶은 목표나 계획이 있으신가요?
----------------------------------------
3번째 대화: '내 이름이 뭐였지?'
AI: 당신의 이름은 김철수입니다. 다른 질문이나 도움이 필요하신 부분이 있으면 말씀해 주세요!
----------------------------------------
4번째 대화: '서울에 살고 있어'
AI: 서울에 살고 계시군요! 서울은 다양한 문화와 맛있는 음식, 멋진 경치가 있는 도시죠. 서울에서 좋아하는 장소나 활동이 있으신가요?
----------------------------------------
5번째 대화: '취미는 축구야'
AI: 축구를 좋아하시군요! 축구는 정말 재미있고 팀워크를 중요시하는 스포츠죠. 어떤 팀을 응원하시나요? 또는 축구를 하시는 것도 좋아하시나요?
----------------------------------------
6번째 대화: '좋아하는 색깔은 파란색이야'
AI: 파란색은 정말 멋진 색깔이죠! 시원하고 안정감을 주는 느낌이 있어요. 파란색과 관련된 특별한 이유가 있나요? 아니면 파란색을 좋아하는 다른 이유가 있으신가요?
----------------------------------------
7번째 대화: '어제 영화를 봤어'
AI: 어떤 영화를 보셨나요? 영화의 장르나 내용이 궁금해요! 재미있었나요?
----------------------------------------
8번째 대화: '오늘은 운동을 했어'
AI: 운동을 하셨군요! 어떤 운동을 하셨나요? 운동 후 기분이 어떠셨나요?
----------------------------------------
9번째 대화: '내 이름이 뭐였지?'
AI: 죄송하지만, 이전 대화에서 이름을

## 4. ConversationSummaryMemory 예시
**오래된 대화를 요약해서 기억하는 똑똑한 챗봇**

모든 대화를 기억하되, 오래된 내용은 요약해서 저장하는 방법입니다!


### 1. AI 모델 설정

In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 2. 요약 기능이 포함된 노드 정의

In [22]:
from langchain_core.messages import SystemMessage

def summarize_old_messages(messages, max_messages=4):
    """오래된 메시지들을 요약하는 함수"""
    if len(messages) <= max_messages:
        return messages
    
    # 오래된 메시지와 최근 메시지 분리
    old_messages = messages[:-max_messages]
    recent_messages = messages[-max_messages:]
    
    # 요약 만들기
    summary_text = "이전 대화 요약:\\n"
    for msg in old_messages:
        if msg.type == "human":
            summary_text += f"- 사용자: {msg.content}\\n"
        elif msg.type == "ai":
            summary_text += f"- AI: {msg.content}\\n"
    
    # 요약을 시스템 메시지로 만들기
    summary_message = SystemMessage(content=summary_text)
    
    return [summary_message] + recent_messages

In [23]:
from langgraph.graph import MessagesState

def chat_node_with_summary(state: MessagesState):
    # 메시지가 너무 많으면 요약
    summarized_messages = summarize_old_messages(state["messages"])
    
    response = llm.invoke(summarized_messages)
    return {"messages": [response]}

### 3. 그래프 생성

In [24]:
from langgraph.graph import StateGraph, MessagesState, START, END 

graph_summary = StateGraph(MessagesState)
graph_summary.add_node("chat", chat_node_with_summary)
graph_summary.add_edge(START, "chat")
graph_summary.add_edge("chat", END)

### 4. 컴파일

In [25]:
from langgraph.checkpoint.memory import MemorySaver

memory_summary = MemorySaver()
app_with_summary = graph_summary.compile(checkpointer=memory_summary)

print("Summary Memory 챗봇이 준비되었습니다! (오래된 대화는 요약해서 기억)")

Summary Memory 챗봇이 준비되었습니다! (오래된 대화는 요약해서 기억)


### 5. 테스트

In [26]:
from langchain_core.messages import HumanMessage
import uuid

# 중요: 같은 thread_id를 사용해야 대화가 연결됩니다!
memory_id = str(uuid.uuid4())
# Summary Memory 테스트
config_summary = {"configurable": {"thread_id": memory_id}}

print("=== Summary Memory 챗봇 테스트 ===")
print("많은 정보를 말해보고, 나중에 요약된 정보로 기억하는지 확인해보겠습니다!\\n")

=== Summary Memory 챗봇 테스트 ===
많은 정보를 말해보고, 나중에 요약된 정보로 기억하는지 확인해보겠습니다!\n


In [27]:
# 테스트할 메시지들
test_messages = [
    "내 이름은 김철수야",
    "나는 서울에 살아", 
    "취미는 축구야",
    "좋아하는 음식은 피자야",
    "오늘 날씨가 좋네",
    "좋아하는 과일은 수박,복숭아야",
    "내 이름과 사는 곳이 뭐였지?"  # 오래된 정보가 요약되어 기억되는지 테스트
]

for i, msg in enumerate(test_messages, 1):
    print(f"{i}번째 대화: '{msg}'")
    result = app_with_summary.invoke(
        {"messages": [HumanMessage(content=msg)]}, 
        config=config_summary
    )
    print(f"AI: {result['messages'][-1].content}")
    print("-" * 40)

print("\\n결과: 오래된 대화는 요약되지만 중요한 정보는 기억합니다!")
print("메모리 효율적이면서도 핵심 정보를 유지해요!")

1번째 대화: '내 이름은 김철수야'
AI: 안녕하세요, 김철수님! 어떻게 도와드릴까요?
----------------------------------------
2번째 대화: '나는 서울에 살아'
AI: 서울에 사시는군요! 서울은 정말 다양한 문화와 매력이 있는 도시죠. 서울에서 좋아하는 장소나 활동이 있으신가요?
----------------------------------------
3번째 대화: '취미는 축구야'
AI: 축구를 좋아하시군요! 축구는 정말 재미있고 팀워크도 중요한 스포츠죠. 어떤 팀을 응원하시나요? 또는 자주 축구를 하시나요?
----------------------------------------
4번째 대화: '좋아하는 음식은 피자야'
AI: 피자를 좋아하시군요! 피자는 다양한 토핑과 스타일로 즐길 수 있어서 정말 매력적인 음식이죠. 어떤 종류의 피자를 가장 좋아하시나요? 마르게리타, 페퍼로니, 아니면 다른 특별한 토핑이 있나요?
----------------------------------------
5번째 대화: '오늘 날씨가 좋네'
AI: 날씨가 좋다니 기분이 좋으시겠어요! 이렇게 좋은 날에는 밖에 나가서 활동하기 좋은 것 같아요. 혹시 오늘 특별한 계획이 있으신가요?
----------------------------------------
6번째 대화: '좋아하는 과일은 수박,복숭아야'
AI: 수박과 복숭아를 좋아하시군요! 여름철에 특히 맛있는 과일들이죠. 수박은 시원하고 상큼해서 더위를 식히기에 좋고, 복숭아는 달콤하고 부드러워서 정말 맛있죠. 두 과일을 함께 먹으면 더 맛있을 것 같아요! 과일을 자주 드시나요?
----------------------------------------
7번째 대화: '내 이름과 사는 곳이 뭐였지?'
AI: 당신의 이름은 김철수님이고, 서울에 사신다고 하셨습니다!
----------------------------------------
\n결과: 오래된 대화는 요약되지만 중요한 정보는 기억합니다

## 5. 암호화된 SQLite Memory 예시 (EncryptedSerializer)
**보안이 중요한 대화를 암호화해서 저장하는 챗봇**

실제 서비스에서는 대화 내용을 안전하게 보관해야 해요. SQLite 데이터베이스에 암호화해서 저장하는 방법입니다!


### 1. AI 모델 설정

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 2. 챗봇 노드 정의

In [29]:
from langgraph.graph import MessagesState

def secure_chat_node(state: MessagesState):
    """보안이 적용된 챗봇 노드"""
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

### 4. 그래프 생성

In [30]:
from langgraph.graph import StateGraph, MessagesState, START, END

secure_graph = StateGraph(MessagesState)
secure_graph.add_node("chat", secure_chat_node)
secure_graph.add_edge(START, "chat")
secure_graph.add_edge("chat", END)

### 5. 암호화 키 설정 (실제 운영에서는 환경변수로 관리!)

In [31]:
import hashlib 
import os

def aes_key(seed_string:str):
    return hashlib.sha256(str(seed_string).encode()).hexdigest()[:32]

seed = "Development" # 개발인 경우 
# 운영인 경우 
if os.getenv("ENV") == "Production":
    seed = os.getenv("SECRET_SEED", "your-secret-seed-here")

os.environ["LANGGRAPH_AES_KEY"] = aes_key(seed)
print("암호화 키가 설정되었습니다!")
print("실제 운영에서는 환경변수나 보안 저장소에서 키를 가져오세요!")

암호화 키가 설정되었습니다!
실제 운영에서는 환경변수나 보안 저장소에서 키를 가져오세요!


### 6. 암호화된 SQLite 저장소 설정

In [32]:
import sqlite3
from langgraph.checkpoint.serde.encrypted import EncryptedSerializer
from langgraph.checkpoint.sqlite import SqliteSaver

# check_same_thread=False 옵션으로 멀티스레드 환경에서 사용 가능하게 설정
def get_db_connection():
    return sqlite3.connect("secure_memory.db", check_same_thread=False)  # SQLite 데이터베이스

serde = EncryptedSerializer.from_pycryptodome_aes()  # 암호화 직렬화기
checkpointer = SqliteSaver(get_db_connection(), serde=serde)  # 암호화된 저장소

### 7. 암호화된 Memory와 함께 컴파일

In [33]:
secure_app = secure_graph.compile(checkpointer=checkpointer)

print("암호화된 SQLite Memory 챗봇이 준비되었습니다!")
print("대화 내용이 'secure_memory.db' 파일에 암호화되어 저장됩니다!")

암호화된 SQLite Memory 챗봇이 준비되었습니다!
대화 내용이 'secure_memory.db' 파일에 암호화되어 저장됩니다!


### 8. 테스트 

In [34]:
import uuid

# 중요: 같은 thread_id를 사용해야 대화가 연결됩니다!
memory_id = str(uuid.uuid4())
# 암호화된 Memory 테스트
config_secure = {"configurable": {"thread_id": memory_id}}

print("=== 암호화된 SQLite Memory 챗봇 테스트 ===")
print("중요한 개인정보를 포함한 대화를 테스트해보겠습니다!\\n")
print(f"config_secure: {config_secure}")

=== 암호화된 SQLite Memory 챗봇 테스트 ===
중요한 개인정보를 포함한 대화를 테스트해보겠습니다!\n
config_secure: {'configurable': {'thread_id': 'ef751c5c-ea70-4afc-a79b-8cbbf40605c3'}}


In [35]:
from langchain_core.messages import HumanMessage

# 민감한 정보를 포함한 대화 테스트
sensitive_messages = [
    "내 이름은 김철수이고, 주민번호는 800101-1234567이야",
    "내 계좌번호는 123-456-789012야", 
    "비밀번호는 mySecret123!이야",
    "내 개인정보가 뭐였지?",
    "계좌번호를 다시 알려줘"
]

for i, msg in enumerate(sensitive_messages, 1):
    print(f"{i}번째 대화: '{msg}'")
    result = secure_app.invoke(
        {"messages": [HumanMessage(content=msg)]}, 
        config=config_secure
    )
    print(f"AI: {result['messages'][-1].content}")
    print("-" * 50)

print("\\n결과: 모든 대화가 암호화되어 안전하게 저장되었습니다!")
print("데이터베이스를 직접 열어봐도 암호화된 내용만 보입니다!")

1번째 대화: '내 이름은 김철수이고, 주민번호는 800101-1234567이야'
AI: 죄송하지만, 개인 정보를 요청하거나 저장할 수 없습니다. 다른 질문이나 도움이 필요하신 부분이 있다면 말씀해 주세요!
--------------------------------------------------
2번째 대화: '내 계좌번호는 123-456-789012야'
AI: 죄송하지만, 개인적인 금융 정보나 민감한 정보를 공유하실 수 없습니다. 다른 질문이나 도움이 필요하신 부분이 있다면 말씀해 주세요!
--------------------------------------------------
3번째 대화: '비밀번호는 mySecret123!이야'
AI: 죄송하지만, 비밀번호와 같은 민감한 정보를 공유하실 수 없습니다. 보안을 위해 개인 정보를 안전하게 보호하는 것이 중요합니다. 다른 질문이나 도움이 필요하신 부분이 있다면 말씀해 주세요!
--------------------------------------------------
4번째 대화: '내 개인정보가 뭐였지?'
AI: 죄송하지만, 저는 개인 정보를 저장하거나 기억할 수 없습니다. 또한, 개인정보 보호를 위해 그러한 정보를 요청하거나 처리하지 않습니다. 다른 질문이나 도움이 필요하신 부분이 있다면 말씀해 주세요!
--------------------------------------------------
5번째 대화: '계좌번호를 다시 알려줘'
AI: 죄송하지만, 개인적인 정보나 계좌번호와 같은 민감한 정보를 저장하거나 제공할 수 없습니다. 보안과 개인정보 보호를 위해 이러한 정보를 안전하게 관리하는 것이 중요합니다. 다른 질문이나 도움이 필요하신 부분이 있다면 말씀해 주세요!
--------------------------------------------------
\n결과: 모든 대화가 암호화되어 안전하게 저장되었습니다!
데이터베이스를 직접 열어봐도 암호화된 내용만 보입니다!


### 9. 데이터베이스 내용 확인 (암호화 확인)

#### 보안 장점 
- 대화 내용이 암호화되어 저장 
- 키 없이는 데이터 복호화 불가능 
- SQLite 파일을 직접 열어도 내용 확인 불가 
- 실제 서비스에 적합한 보안 수준 

In [ ]:
print("\\n=== 데이터베이스 암호화 확인 ===")

# SQLite 데이터베이스에서 실제 저장된 데이터 확인
cursor = get_db_connection().cursor()

try:
    # 테이블 구조 확인
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    print(f"생성된 테이블들: {tables}")
    
    # 실제 데이터 일부 확인 (암호화된 상태)
    cursor.execute("SELECT * FROM checkpoints LIMIT 1")
    encrypted_data = cursor.fetchone()
    
    if encrypted_data:
        print("\\n데이터베이스에 저장된 실제 데이터 (일부):")
        print(f"Thread ID: {encrypted_data[0] if encrypted_data[0] else 'N/A'}")
        print(f"암호화된 데이터 (처음 100자): {str(encrypted_data)[:100]}...")
        print("\\n보시다시피 실제 대화 내용은 암호화되어 읽을 수 없습니다!")
    else:
        print("아직 저장된 데이터가 없습니다.")
        
except Exception as e:
    print(f"데이터베이스 확인 중 오류: {e}")


\n=== 데이터베이스 암호화 확인 ===
생성된 테이블들: [('checkpoints',), ('writes',)]
\n데이터베이스에 저장된 실제 데이터 (일부):
Thread ID: ef751c5c-ea70-4afc-a79b-8cbbf40605c3
암호화된 데이터 (처음 100자): ('ef751c5c-ea70-4afc-a79b-8cbbf40605c3', '', '1f07e6bc-e171-6088-bfff-5452d229018d', None, 'msgpack+...
\n보시다시피 실제 대화 내용은 암호화되어 읽을 수 없습니다!
\n보안 장점:
1. 대화 내용이 암호화되어 저장
2. 키 없이는 데이터 복호화 불가능
3. SQLite 파일을 직접 열어도 내용 확인 불가
4. 실제 서비스에 적합한 보안 수준


### EncryptedSerializer 사용 시 주의사항

#### 필수 설치 패키지
```bash
pip install pycryptodome  # 암호화를 위해 필요
```

#### 암호화 키 관리
```python
# 나쁜 예시 - 코드에 직접 하드코딩
os.environ["LANGGRAPH_AES_KEY"] = "hardcoded-key"

# 좋은 예시 - 환경변수나 보안 저장소 사용
import os
key = os.getenv("LANGGRAPH_AES_KEY")  # 환경변수에서 가져오기
```

#### 실제 운영 환경에서의 권장사항
1. **환경변수로 키 관리**: 코드에 키를 하드코딩하지 말 것
2. **키 회전 정책**: 정기적으로 암호화 키 변경
3. **백업 정책**: 암호화된 데이터베이스 백업 계획
4. **접근 권한**: 데이터베이스 파일 접근 권한 제한


# 업데이트된 Memory 종류별 정리

| Memory 종류 | 장점 | 단점 | 언제 사용할까? | 보안 수준 |
|------------|------|------|----------------|-----------|
| **Memory 없음** | 빠름, 단순함 | 대화 기억 안함 | 일회성 질문답변 | 보통 |
| **MemorySaver** | 모든 대화 기억 | 메모리 많이 사용 | 짧은 대화, 모든 내용 중요 | 보통 |
| **WindowMemory** | 적당한 메모리 사용 | 오래된 내용 잊음 | 긴 대화, 최근 내용만 중요 | 보통 |
| **SummaryMemory** | 효율적, 핵심 유지 | 요약 과정 필요 | 긴 대화, 핵심 정보 유지 | 보통 |
| **EncryptedSQLite** | 영구저장, 보안 | 설정 복잡, 성능 오버헤드 | 운영 서비스, 개인정보 포함 | **높음** |

### 실제 사용 시나리오별 추천

#### 개발/테스트 환경
- **MemorySaver**: 간단한 테스트용
- **WindowMemory**: 성능 테스트용

#### 운영 환경  
- **EncryptedSQLite**: 개인정보/민감정보 포함 서비스
- **SummaryMemory + SQLite**: 일반 서비스

#### 모바일/임베디드
- **WindowMemory**: 메모리 제약 환경
- **InMemory**: 일시적 사용
